In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (16901 Collins Ave #4105, Sunny Isles Beach, F...
2     (788 NE 23rd St #3701, Miami, FL 33137, USA, (...
3     (650 NE 32nd St, Miami, FL 33137, USA, (25.806...
4     (450 Alton Rd APT 2603, Miami Beach, FL 33139,...
5     (2000 S Bayshore Dr APT 50, Coconut Grove, FL ...
6     (801 S Miami Ave, Miami, FL 33131, USA, (25.76...
7     (1900 Sunset Harbour Dr #1805, Miami Beach, FL...
8     (2201 Collins Ave #1215, Miami Beach, FL 33139...
9     (90 SW 3rd St PH 15, Miami, FL 33130, USA, (25...
10    (1000 Brickell Plaza, Miami, FL 33131, USA, (2...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Jade Signature\r16901 Collins Ave 4105\rSunny ...,2/17/2022,10/24/2022,224,"$7,000,000.00",4781,"$1,844.53",Marissa Everhart,Florida Realty of Miami Corp,Luis Paredes,Million and Up Realty,Jade Signature 16901 Collins Ave 4105 Sunny Is...,Jade Signature,16901 Collins Ave 4105 Sunny Isles Beach,"(16901 Collins Ave #4105, Sunny Isles Beach, F...","(25.9328476, -80.1211362, 0.0)",25.932848,-80.121136,0.0
2,The Elysee Condo\r788 NE 23rd St 3701\rMiami,2/15/2022,10/24/2022,58,"$4,100,000.00",3471,"$1,181.22",Sandra Masis,Cervera Real Estate Inc.,David Siddons,Douglas Elliman,The Elysee Condo 788 NE 23rd St 3701 Miami,The Elysee Condo,788 NE 23rd St 3701 Miami,"(788 NE 23rd St #3701, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0
3,Paraiso Bay\r650 NE 32nd St PH5201\rMiami,10/1/2022,10/28/2022,1,"$3,415,500.00",4210,$811.28,Alejandra Castillo,PMG Residential LLC,Leila Abou Jokh,"Related ISG Realty, LLC.",Paraiso Bay 650 NE 32nd St PH5201 Miami,Paraiso Bay,650 NE 32nd St PH5201 Miami,"(650 NE 32nd St, Miami, FL 33137, USA, (25.806...","(25.8068747, -80.1862424, 0.0)",25.806875,-80.186242,0.0
4,Icon\r450 Alton Rd 2603\rMiami Beach,6/29/2022,10/27/2022,107,"$2,925,000.00",2158,"$1,355.42",Laurent Benzaquen,Cromwell & Forbes Inc,Adrian Burke,One Sotheby's International Realty,Icon 450 Alton Rd 2603 Miami Beach,Icon,450 Alton Rd 2603 Miami Beach,"(450 Alton Rd APT 2603, Miami Beach, FL 33139,...","(25.7719135, -80.1380353, 0.0)",25.771914,-80.138035,0.0
5,L'Hermitage\r2000 S Bayshore Dr 50\rMiami,7/19/2022,10/26/2022,57,"$2,750,000.00",4535,$738.26,Francis Pimentel de Matos,"Boschetti Realty Group, LLC.",Beatriz Pesquera,"AG Real Estate Advisors, LLC",L'Hermitage 2000 S Bayshore Dr 50 Miami,L'Hermitage,2000 S Bayshore Dr 50 Miami,"(2000 S Bayshore Dr APT 50, Coconut Grove, FL ...","(25.7362472, -80.22459429999999, 0.0)",25.736247,-80.224594,0.0
6,SLS Lux\r801 S Miami Ave LPH5602\rMiami,5/20/2022,10/24/2022,144,"$2,160,000.00",2177,$992.19,Alexandar Alexandrov,"Compass Florida, LLC.",Alexandar Alexandrov,"Compass Florida, LLC.",SLS Lux 801 S Miami Ave LPH5602 Miami,SLS Lux,801 S Miami Ave LPH5602 Miami,"(801 S Miami Ave, Miami, FL 33131, USA, (25.76...","(25.765952, -80.1930337, 0.0)",25.765952,-80.193034,0.0
7,Sunset Harbour North Condo\r1900 Sunset Harbou...,7/27/2022,10/25/2022,62,"$2,150,000.00",1930,"$1,113.99",Terrence Segall,Brown Harris Stevens,Ingrid Santos,"Compass Florida, LLC.",Sunset Harbour North Condo 1900 Sunset Harbour...,Sunset Harbour North Condo,1900 Sunset Harbour Dr 1805 Miami Beach,"(1900 Sunset Harbour Dr #1805, Miami Beach, FL...","(25.7950664, -80.14463839999999, 0.0)",25.795066,-80.144638,0.0
8,Collins Ave Condo\r2201 Collins Ave 1215\rMiam...,6/16/2022,10/27/2022,97,"$2,100,000.00",1026,"$2,046.78",Lisa Van Wagenen,Brown Harris Stevens,Ida Schwartz,"Compass Florida, LLC.",Collins Ave Condo 2201 Collins Ave 1215 Miami ...,Collins Ave Condo,2201 Collins Ave 1215 Miami Beach,"(2201 Collins Ave #1215, Miami Beach, FL 33139...","(25.7975046, -80.1281362, 0.0)",25.797505,-80.128136,0.0
9,Riverfront\r90 SW 3rd St PH-15\rMiami,8/15/2022,10/27/2022,43,"$1,870,000.00",2246,$832.59,Denise Gomm Santos PA,"Compass Florida, LLC.",Denise Gomm Santos PA,"Compass Florida, LLC.",Riverfront 90 SW 3rd St PH-15 Miami,Riverfront,90 SW 3rd St PH-15 Miami,"(90 SW 3rd St PH 15, Miami, FL 33130, USA, (25...","(25.7703098, -80.19520179999999, 0.0)",25.770310,-80.195202,0.0
10,B0rickell Flatiron\r1000 Brickell Plz PH6006\r...,6/13/2022,10/26/2022,116,"$1,525,000.00",1249,"$1,220.98",Aramis Ibarra Alvarez,Florida Capital Realty,Christina Morenza,Florida One Realty,B0rickell Flatiron 1000 Brickell Plz PH6006 Miami,B,0rickell Flatiron 1000 Brickell Plz PH6006 Miami,"(1000 Brickell Plaza, Miami

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [15]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Jade Signature\r16901 Collins Ave 4105\rSunny ...,2/17/2022,10/24/2022,224,"$7,000,000.00",4781,"$1,844.53",Marissa Everhart,Florida Realty of Miami Corp,Luis Paredes,Million and Up Realty,Jade Signature 16901 Collins Ave 4105 Sunny Is...,Jade Signature,16901 Collins Ave 4105 Sunny Isles Beach,"(16901 Collins Ave #4105, Sunny Isles Beach, F...","(25.9328476, -80.1211362, 0.0)",25.932848,-80.121136,0.0
2,The Elysee Condo\r788 NE 23rd St 3701\rMiami,2/15/2022,10/24/2022,58,"$4,100,000.00",3471,"$1,181.22",Sandra Masis,Cervera Real Estate Inc.,David Siddons,Douglas Elliman,The Elysee Condo 788 NE 23rd St 3701 Miami,The Elysee Condo,788 NE 23rd St 3701 Miami,"(788 NE 23rd St #3701, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0
3,Paraiso Bay\r650 NE 32nd St PH5201\rMiami,10/1/2022,10/28/2022,1,"$3,415,500.00",4210,$811.28,Alejandra Castillo,PMG Residential LLC,Leila Abou Jokh,"Related ISG Realty, LLC.",Paraiso Bay 650 NE 32nd St PH5201 Miami,Paraiso Bay,650 NE 32nd St PH5201 Miami,"(650 NE 32nd St, Miami, FL 33137, USA, (25.806...","(25.8068747, -80.1862424, 0.0)",25.806875,-80.186242,0.0
4,Icon\r450 Alton Rd 2603\rMiami Beach,6/29/2022,10/27/2022,107,"$2,925,000.00",2158,"$1,355.42",Laurent Benzaquen,Cromwell & Forbes Inc,Adrian Burke,One Sotheby's International Realty,Icon 450 Alton Rd 2603 Miami Beach,Icon,450 Alton Rd 2603 Miami Beach,"(450 Alton Rd APT 2603, Miami Beach, FL 33139,...","(25.7719135, -80.1380353, 0.0)",25.771914,-80.138035,0.0
5,L'Hermitage\r2000 S Bayshore Dr 50\rMiami,7/19/2022,10/26/2022,57,"$2,750,000.00",4535,$738.26,Francis Pimentel de Matos,"Boschetti Realty Group, LLC.",Beatriz Pesquera,"AG Real Estate Advisors, LLC",L'Hermitage 2000 S Bayshore Dr 50 Miami,L'Hermitage,2000 S Bayshore Dr 50 Miami,"(2000 S Bayshore Dr APT 50, Coconut Grove, FL ...","(25.7362472, -80.22459429999999, 0.0)",25.736247,-80.224594,0.0
6,SLS Lux\r801 S Miami Ave LPH5602\rMiami,5/20/2022,10/24/2022,144,"$2,160,000.00",2177,$992.19,Alexandar Alexandrov,"Compass Florida, LLC.",Alexandar Alexandrov,"Compass Florida, LLC.",SLS Lux 801 S Miami Ave LPH5602 Miami,SLS Lux,801 S Miami Ave LPH5602 Miami,"(801 S Miami Ave, Miami, FL 33131, USA, (25.76...","(25.765952, -80.1930337, 0.0)",25.765952,-80.193034,0.0
7,Sunset Harbour North Condo\r1900 Sunset Harbou...,7/27/2022,10/25/2022,62,"$2,150,000.00",1930,"$1,113.99",Terrence Segall,Brown Harris Stevens,Ingrid Santos,"Compass Florida, LLC.",Sunset Harbour North Condo 1900 Sunset Harbour...,Sunset Harbour North Condo,1900 Sunset Harbour Dr 1805 Miami Beach,"(1900 Sunset Harbour Dr #1805, Miami Beach, FL...","(25.7950664, -80.14463839999999, 0.0)",25.795066,-80.144638,0.0
8,Collins Ave Condo\r2201 Collins Ave 1215\rMiam...,6/16/2022,10/27/2022,97,"$2,100,000.00",1026,"$2,046.78",Lisa Van Wagenen,Brown Harris Stevens,Ida Schwartz,"Compass Florida, LLC.",Collins Ave Condo 2201 Collins Ave 1215 Miami ...,Collins Ave Condo,2201 Collins Ave 1215 Miami Beach,"(2201 Collins Ave #1215, Miami Beach, FL 33139...","(25.7975046, -80.1281362, 0.0)",25.797505,-80.128136,0.0
9,Riverfront\r90 SW 3rd St PH-15\rMiami,8/15/2022,10/27/2022,43,"$1,870,000.00",2246,$832.59,Denise Gomm Santos PA,"Compass Florida, LLC.",Denise Gomm Santos PA,"Compass Florida, LLC.",Riverfront 90 SW 3rd St PH-15 Miami,Riverfront,90 SW 3rd St PH-15 Miami,"(90 SW 3rd St PH 15, Miami, FL 33130, USA, (25...","(25.7703098, -80.19520179999999, 0.0)",25.770310,-80.195202,0.0
10,B0rickell Flatiron\r1000 Brickell Plz PH6006\r...,6/13/2022,10/26/2022,116,"$1,525,000.00",1249,"$1,220.98",Aramis Ibarra Alvarez,Florida Capital Realty,Christina Morenza,Florida One Realty,B0rickell Flatiron 1000 Brickell Plz PH6006 Miami,Brickell Flatiron,1000 Brickell Plaza,"(1000 Brickell Plaza, Miami, FL 33131, U

In [16]:
df.at[2,'building_name']=('Elysee Miami')

df.at[10,'building_name']=('Brickell Flatiron')
df.at[10,'address_only']=('1000 Brickell Plaza')

In [13]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [17]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('October 23rd - October 29th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [18]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_110122
